## 用estimator 改写 mnist
https://host02.katacoda.com/basiafusinska/courses/tensorflow-in-3-sentences/tensorflow-mnist-estimators

### 简单的单层dnn网络

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.ERROR)

image_size = 28
labels_size = 10
hidden_size = 1024

# Read in the MNIST dataset
mnist = input_data.read_data_sets("/home/w/tmp/tensorflow/mnist/input_data", one_hot=False)

def input_fn(dataset):
    features = dataset.images
    labels = dataset.labels.astype(np.int32)
    return features, labels

# Define the Estimator
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=image_size*image_size)]
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[hidden_size],
                                              n_classes=labels_size,
                                              optimizer=tf.train.AdamOptimizer())

# Fit the model
features, labels = input_fn(mnist.train)
classifier.fit(x=features, y=labels, batch_size=100, steps=1000)

# Evaluate the model on the test data
features, labels = input_fn(mnist.test)
test_accuracy = classifier.evaluate(x=features, y=labels, steps=1)["accuracy"]

print("\nTest accuracy: %g %%"%(test_accuracy*100))

# Predict the new examples and compare with the onderlying values
features = mnist.validation.images[:10]
labels = mnist.validation.labels[:10].astype(np.int32)
predictions = list(classifier.predict(x=features))

print("\nPredicted labels from validation set: %s"%predictions)
print("Underlying values: %s"%list(labels))


### 复杂的cnn网络
https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/examples/tutorials/layers/cnn_mnist.py

In [1]:
import numpy as np
import tensorflow as tf

# 限制显卡内存
config = tf.ConfigProto(allow_soft_placement=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors 
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer 
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer 
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [2]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, 
        model_dir="/home/w/tmp/tensorflow/mnist_convnet_model")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_data},
  y=train_labels,
  batch_size=100,
  num_epochs=None,
  shuffle=True)
mnist_classifier.train(
  input_fn=train_input_fn,
  steps=20000,
  hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": eval_data},
  y=eval_labels,
  num_epochs=1,
  shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/w/tmp/tensorflow/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt-20003
INFO:tensorflow:Saving checkpoints for 20004 into /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00036716  0.0028864   0.00004472  0.00163343  0.07248198  0.00742182
   0.00003236  0.00626353  0.00069164  0.90817696]
 [ 0.0000009   0.00000207  0.000

INFO:tensorflow:loss = 0.147521, step = 20004
INFO:tensorflow:global_step/sec: 179.261
INFO:tensorflow:loss = 0.168858, step = 20104 (0.542 sec)
INFO:tensorflow:global_step/sec: 211.871
INFO:tensorflow:loss = 0.0711041, step = 20204 (0.472 sec)
INFO:tensorflow:global_step/sec: 213.086
INFO:tensorflow:loss = 0.227588, step = 20304 (0.470 sec)
INFO:tensorflow:global_step/sec: 211.369
INFO:tensorflow:loss = 0.221463, step = 20404 (0.473 sec)
INFO:tensorflow:global_step/sec: 210.86
INFO:tensorflow:loss = 0.123198, step = 20504 (0.474 sec)
INFO:tensorflow:global_step/sec: 211.651
INFO:tensorflow:loss = 0.228068, step = 20604 (0.472 sec)
INFO:tensorflow:global_step/sec: 213.805
INFO:tensorflow:loss = 0.0868666, step = 20704 (0.468 sec)
INFO:tensorflow:global_step/sec: 216.139
INFO:tensorflow:loss = 0.0766645, step = 20804 (0.463 sec)
INFO:tensorflow:global_step/sec: 215.26
INFO:tensorflow:loss = 0.22642, step = 20904 (0.464 sec)
INFO:tensorflow:global_step/sec: 209.824
INFO:tensorflow:loss =

INFO:tensorflow:loss = 0.0964704, step = 25004 (0.484 sec)
INFO:tensorflow:global_step/sec: 210.764
INFO:tensorflow:loss = 0.0590065, step = 25104 (0.460 sec)
INFO:tensorflow:global_step/sec: 216.147
INFO:tensorflow:loss = 0.0750521, step = 25204 (0.462 sec)
INFO:tensorflow:global_step/sec: 206.558
INFO:tensorflow:loss = 0.0896906, step = 25304 (0.484 sec)
INFO:tensorflow:global_step/sec: 211.64
INFO:tensorflow:loss = 0.187506, step = 25404 (0.474 sec)
INFO:tensorflow:global_step/sec: 213.762
INFO:tensorflow:loss = 0.178361, step = 25504 (0.468 sec)
INFO:tensorflow:global_step/sec: 213.542
INFO:tensorflow:loss = 0.0760943, step = 25604 (0.467 sec)
INFO:tensorflow:global_step/sec: 217
INFO:tensorflow:loss = 0.0846064, step = 25704 (0.461 sec)
INFO:tensorflow:global_step/sec: 215.736
INFO:tensorflow:loss = 0.160039, step = 25804 (0.464 sec)
INFO:tensorflow:global_step/sec: 217
INFO:tensorflow:loss = 0.211431, step = 25904 (0.461 sec)
INFO:tensorflow:global_step/sec: 209.311
INFO:tensorfl

INFO:tensorflow:loss = 0.117601, step = 30004 (0.489 sec)
INFO:tensorflow:global_step/sec: 204.164
INFO:tensorflow:loss = 0.135572, step = 30104 (0.475 sec)
INFO:tensorflow:global_step/sec: 218.166
INFO:tensorflow:loss = 0.173473, step = 30204 (0.458 sec)
INFO:tensorflow:global_step/sec: 212.29
INFO:tensorflow:loss = 0.0499308, step = 30304 (0.472 sec)
INFO:tensorflow:global_step/sec: 213.398
INFO:tensorflow:loss = 0.116598, step = 30404 (0.469 sec)
INFO:tensorflow:global_step/sec: 210.502
INFO:tensorflow:loss = 0.0395054, step = 30504 (0.474 sec)
INFO:tensorflow:global_step/sec: 211.271
INFO:tensorflow:loss = 0.0844747, step = 30604 (0.473 sec)
INFO:tensorflow:global_step/sec: 212.415
INFO:tensorflow:loss = 0.082434, step = 30704 (0.471 sec)
INFO:tensorflow:global_step/sec: 217.111
INFO:tensorflow:loss = 0.138741, step = 30804 (0.461 sec)
INFO:tensorflow:global_step/sec: 215.921
INFO:tensorflow:loss = 0.0459335, step = 30904 (0.463 sec)
INFO:tensorflow:global_step/sec: 217.377
INFO:te

INFO:tensorflow:loss = 0.0661207, step = 35004 (0.472 sec)
INFO:tensorflow:global_step/sec: 209.876
INFO:tensorflow:loss = 0.109769, step = 35104 (0.461 sec)
INFO:tensorflow:global_step/sec: 212.19
INFO:tensorflow:loss = 0.10192, step = 35204 (0.471 sec)
INFO:tensorflow:global_step/sec: 211.886
INFO:tensorflow:loss = 0.127737, step = 35304 (0.471 sec)
INFO:tensorflow:global_step/sec: 215.691
INFO:tensorflow:loss = 0.0419498, step = 35404 (0.465 sec)
INFO:tensorflow:global_step/sec: 217.583
INFO:tensorflow:loss = 0.216545, step = 35504 (0.459 sec)
INFO:tensorflow:global_step/sec: 211.811
INFO:tensorflow:loss = 0.0989907, step = 35604 (0.472 sec)
INFO:tensorflow:global_step/sec: 218.053
INFO:tensorflow:loss = 0.0496074, step = 35704 (0.460 sec)
INFO:tensorflow:global_step/sec: 209.432
INFO:tensorflow:loss = 0.0964368, step = 35804 (0.477 sec)
INFO:tensorflow:global_step/sec: 218.507
INFO:tensorflow:loss = 0.088836, step = 35904 (0.458 sec)
INFO:tensorflow:global_step/sec: 210.543
INFO:te

INFO:tensorflow:Starting evaluation at 2017-09-04-05:17:41
INFO:tensorflow:Restoring parameters from /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt-40003
INFO:tensorflow:Finished evaluation at 2017-09-04-05:17:42
INFO:tensorflow:Saving dict for global step 40003: accuracy = 0.9801, global_step = 40003, loss = 0.064799
{'accuracy': 0.98009998, 'loss': 0.064799033, 'global_step': 40003}


## 练习版

### 简单的单层dnn网络

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.ERROR)

# Read in the MNIST dataset
mnist = input_data.read_data_sets("/home/w/tmp/tensorflow/mnist/input_data", one_hot=False)


Extracting /home/w/tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /home/w/tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /home/w/tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /home/w/tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz


In [22]:
# define estimator 
classifier=tf.contrib.learn.DNNClassifier(
                    feature_columns=[tf.contrib.layers.real_valued_column('',dimension=28*28)],
                    hidden_units=[1024],
                    n_classes=10,
                    optimizer=tf.train.AdamOptimizer())

# training
classifier.fit(x=mnist.train.images,
                   y=mnist.train.labels.astype(np.int32),
                   batch_size=100,
                   steps=1000)

# test
classifier.evaluate(x=mnist.test.images,
                   y=mnist.test.labels.astype(np.int32),
                   steps=1)

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x00000214801C5D68>, 'hidden_units': [1024], 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': <tensorflow.python.training.adam.AdamOptimizer object at 0x00000214801C5D30>, 'activation_fn': <function relu at 0x000002122169A048>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

{'accuracy': 0.97390002, 'global_step': 1000, 'loss': 0.083541602}

### 复杂的cnn网络
https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/examples/tutorials/layers/cnn_mnist.py

In [1]:
import numpy as np
import tensorflow as tf

# 限制显卡内存
config = tf.ConfigProto(allow_soft_placement=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
    def conv_layer(inputs, filters, kernel_size=[5, 5], padding='same', activation=tf.nn.relu):
        return tf.layers.conv2d(
            inputs=inputs,
            filters=filters,
            kernel_size=kernel_size,
            padding=padding,
            activation=activation)

    def pool_layer(inputs,
                   pool_size=[2, 2],
                   strides=2):
        return tf.layers.max_pooling2d(
            inputs=inputs,
            pool_size=pool_size,
            strides=strides)

    # input
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])

    # conv 1
    conv1 = conv_layer(input_layer, 32)

    # pooling layer 1
    pool1 = pool_layer(conv1)

    # conv 2
    conv2 = conv_layer(pool1, 64)

    # pooling layer 2
    pool2 = pool_layer(conv2)

    # func 1
    pool2_flat = tf.reshape(pool2, shape=[-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # drop out
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # output
    logits = tf.layers.dense(inputs=dropout, units=10)

    # 以上是普通cnn结构，但用estimator，还需要定义 loss，optimizer,prediction
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # loss(both train,eval) @这个定义还必须在 predict 之后！否则会出错，感觉很诡异啊！！！
    loss = tf.losses.softmax_cross_entropy(logits=logits,
                                           onehot_labels=tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10))
    #
    if mode == tf.estimator.ModeKeys.TRAIN:  # optimizer(only for train)
        op = tf.train.GradientDescentOptimizer(0.001).minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=op)

    if mode == tf.estimator.ModeKeys.EVAL:
        metric_op = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=metric_op)


# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Create the Estimator
classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/home/w/tmp/tensorflow/mnist_convnet_model")

# Set up logging for predictions
logging_hook = tf.train.LoggingTensorHook(tensors={"probabilities": "softmax_tensor"}, every_n_iter=500)

#  train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
classifier.train(input_fn=train_input_fn,
                 steps=500,
                 hooks=[logging_hook])

# evaluate model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_v = classifier.evaluate(input_fn=eval_input_fn)
print('evaluate:', eval_v)

# prediction test data
prediction_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': eval_data[0:10]},
    num_epochs=1,
    shuffle=False)
predic_result = classifier.predict(input_fn=prediction_input_fn)
predic_class = [x['classes'] for x in predic_result]

for i in range(10):
    print('预测结果：', predic_class[i], ' 真实结果：', eval_labels[i])


Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/w/tmp/tensorflow/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt-67003
INFO:tensorflow:Saving checkpoints for 67004 into /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.          0.00000002  0.00000285  0.99995327  0.          0.00000002
   0.          0.          0.00004387  0.00000001]
 [ 0.99995196  0.          0.   

INFO:tensorflow:loss = 0.0830458, step = 67004
INFO:tensorflow:global_step/sec: 184.219
INFO:tensorflow:loss = 0.00364095, step = 67104 (0.526 sec)
INFO:tensorflow:global_step/sec: 214.85
INFO:tensorflow:loss = 0.0128426, step = 67204 (0.465 sec)
INFO:tensorflow:global_step/sec: 214.945
INFO:tensorflow:loss = 0.0529519, step = 67304 (0.465 sec)
INFO:tensorflow:global_step/sec: 215.951
INFO:tensorflow:loss = 0.0475676, step = 67404 (0.464 sec)
INFO:tensorflow:Saving checkpoints for 67503 into /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Loss for final step: 0.051531.


INFO:tensorflow:Starting evaluation at 2017-09-04-14:34:12
INFO:tensorflow:Restoring parameters from /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt-67503
INFO:tensorflow:Finished evaluation at 2017-09-04-14:34:12
INFO:tensorflow:Saving dict for global step 67503: accuracy = 0.9842, global_step = 67503, loss = 0.0469145
evaluate: {'accuracy': 0.9842, 'loss': 0.046914466, 'global_step': 67503}
INFO:tensorflow:Restoring parameters from /home/w/tmp/tensorflow/mnist_convnet_model\model.ckpt-67503
预测结果： 7  真实结果： 7
预测结果： 2  真实结果： 2
预测结果： 1  真实结果： 1
预测结果： 0  真实结果： 0
预测结果： 4  真实结果： 4
预测结果： 1  真实结果： 1
预测结果： 4  真实结果： 4
预测结果： 9  真实结果： 9
预测结果： 5  真实结果： 5
预测结果： 9  真实结果： 9


In [9]:
?tf.metrics

In [10]:
?tf.metrics.accuracy

In [4]:
? tf.estimator.EstimatorSpec

In [4]:
? tf.estimator.Estimator.predict

In [13]:
?tf.estimator.inputs.numpy_input_fn